In [9]:
import tensorflow as tf
import numpy as np

def cost_function(real, pred):
    return tf.reduce_mean(-tf.reduce_sum(real * tf.math.log(pred), axis=1))


x_data = np.array([[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]], dtype=np.float32)
#One-hot encoding
y_data = np.array([[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]], dtype=np.float32) 

X = tf.constant(x_data, dtype=tf.float32)
Y = tf.constant(y_data, dtype=tf.float32)
nb_classes = 3

W = tf.Variable(tf.random.normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')

optimizer = tf.optimizers.SGD(learning_rate=0.1)

for step in range(2001):
    with tf.GradientTape() as tape:
        hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
        cost_val = cost_function(Y, hypothesis)
    grads = tape.gradient(cost_val, [W, b])
    optimizer.apply_gradients(zip(grads, [W, b]))
    if step % 200 == 0:
        print(step, cost_val.numpy())

print('--------------')
# arg_max를 통해 가장 큰 값을 알려줌
def predict(x):
    hypothesis = tf.nn.softmax(tf.matmul(x, W) + b)
    return hypothesis, tf.argmax(hypothesis, 1)

x_test = np.array([[1, 11, 7, 9]], dtype=np.float32)
hypothesis, result = predict(x_test)
print(hypothesis.numpy(), result.numpy())

print('--------------')
x_test = np.array([[1, 3, 4, 3]], dtype=np.float32)
hypothesis, result = predict(x_test)
print(hypothesis.numpy(), result.numpy())

print('--------------')
x_test = np.array([[1, 1, 0, 1]], dtype=np.float32)
hypothesis, result = predict(x_test)
print(hypothesis.numpy(), result.numpy())

print('--------------')
x_test = np.array([[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]], dtype=np.float32)
hypothesis, result = predict(x_test)
print(hypothesis.numpy(), result.numpy())


0 6.0906534
200 0.48276383
400 0.39512897
600 0.3192333
800 0.24957132
1000 0.2230172
1200 0.20265734
1400 0.1856482
1600 0.17121355
1800 0.15880924
2000 0.14803734
--------------
[[1.862870e-03 9.981286e-01 8.578780e-06]] [1]
--------------
[[0.91474044 0.07704637 0.00821323]] [0]
--------------
[[8.5131875e-09 2.9711172e-04 9.9970287e-01]] [2]
--------------
[[1.8628681e-03 9.9812859e-01 8.5787797e-06]
 [9.1474044e-01 7.7046402e-02 8.2132267e-03]
 [8.5131875e-09 2.9711172e-04 9.9970287e-01]] [1 0 2]


In [17]:
import tensorflow as tf
import numpy as np

def cost_function(real, pred):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=real, logits=pred))

xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]].astype(np.int32)

print(x_data.shape, y_data.shape)

nb_classes = 7  # 0 ~ 6

X = tf.constant(x_data, dtype=tf.float32)
Y = tf.constant(y_data, dtype=tf.int32)

# One-hot encoding
Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) #-1은 알아서 넣으라는 의미

W = tf.Variable(tf.random.normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')

optimizer = tf.optimizers.SGD(learning_rate=0.1)

# Calculate accuracy
def get_accuracy(real, pred):
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(real, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

# Train the model with accuracy
for step in range(2001):
    with tf.GradientTape() as tape:
        logits = tf.matmul(X, W) + b    #중간에 넘어오는 값(scores)
        hypothesis = tf.nn.softmax(logits)
        cost_val = cost_function(Y_one_hot, logits)
    grads = tape.gradient(cost_val, [W, b])
    optimizer.apply_gradients(zip(grads, [W, b]))
    accuracy_val = get_accuracy(Y_one_hot, hypothesis)

    if step % 100 == 0:
        print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val.numpy(), accuracy_val.numpy()))


# Make predictions
def predict(x):
    logits = tf.matmul(x, W) + b
    hypothesis = tf.nn.softmax(logits)
    return tf.argmax(hypothesis, 1)

# Calculate accuracy
prediction = predict(X)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Accuracy: {:.2%}".format(accuracy.numpy()))

# Compare predictions and true values
pred = prediction.numpy()
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


(101, 16) (101, 1)
Step:     0	Cost: 6.972	Acc: 2.97%
Step:   100	Cost: 0.681	Acc: 81.19%
Step:   200	Cost: 0.385	Acc: 89.11%
Step:   300	Cost: 0.285	Acc: 93.07%
Step:   400	Cost: 0.229	Acc: 95.05%
Step:   500	Cost: 0.193	Acc: 97.03%
Step:   600	Cost: 0.167	Acc: 97.03%
Step:   700	Cost: 0.147	Acc: 97.03%
Step:   800	Cost: 0.131	Acc: 97.03%
Step:   900	Cost: 0.118	Acc: 97.03%
Step:  1000	Cost: 0.107	Acc: 98.02%
Step:  1100	Cost: 0.098	Acc: 100.00%
Step:  1200	Cost: 0.090	Acc: 100.00%
Step:  1300	Cost: 0.084	Acc: 100.00%
Step:  1400	Cost: 0.078	Acc: 100.00%
Step:  1500	Cost: 0.073	Acc: 100.00%
Step:  1600	Cost: 0.069	Acc: 100.00%
Step:  1700	Cost: 0.065	Acc: 100.00%
Step:  1800	Cost: 0.061	Acc: 100.00%
Step:  1900	Cost: 0.058	Acc: 100.00%
Step:  2000	Cost: 0.055	Acc: 100.00%
Accuracy: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Predi